# **3. Performing Vector Search**
Search the uploaded pdf file to find its most simillar chunck to the input query.

In [1]:
from dotenv import load_dotenv,dotenv_values
import json
import os
from sentence_transformers import SentenceTransformer

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  


load_dotenv()
values_env = dotenv_values(r"C:\Users\Seyed Barabadi\Downloads\Gen AI\AZURE-AI-VECTOR-SEARCH-main\azure_ai_vector_search\notebooks\keys.env")
service_endpoint = values_env["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = values_env["AZURE_SEARCH_INDEX_NAME"]
key = values_env["AZURE_SEARCH_ADMIN_KEY"]
model_name = values_env["MODEL_NAME"]
print(index_name, model_name)


biology all-MiniLM-L6-v2


### We use `SentenceTransformer` form Hugging Face to convert input query to a vecteor

In [5]:
model = SentenceTransformer(model_name)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [7]:
query = "Segmentation"  
query_vector = model.encode([query])[0]

print(query_vector)
len(query_vector)

[ 2.30356194e-02  1.26671875e-02  4.44920026e-02 -3.89090143e-02
 -9.84341558e-03 -2.77222916e-02  1.16374210e-01 -3.10633462e-02
  7.35344645e-03 -7.42051452e-02  2.16113124e-02 -2.92580985e-02
 -2.43248660e-02  3.65417451e-02 -4.89201546e-02 -3.33300680e-02
 -3.51171531e-02  7.97075629e-02 -1.05214193e-01 -3.67749780e-02
 -7.42150238e-03 -1.16393277e-02 -6.64043725e-02 -1.18388962e-02
  4.79649901e-02  8.82574022e-02  3.63940559e-02  6.97667524e-03
  5.70957623e-02 -5.15103042e-02  3.25405896e-02  5.39210290e-02
  1.31803915e-01  1.92847177e-02 -1.00792237e-02 -4.30897810e-02
  2.85607874e-02  4.23089713e-02  4.69758129e-03  1.62016265e-02
 -9.48535204e-02 -4.78087626e-02  5.15713356e-03 -1.60939023e-02
  7.83037022e-02  2.15603933e-02 -5.93567640e-03  2.50713248e-02
  4.32969518e-02  1.07064238e-02 -7.33178556e-02 -3.93874943e-02
 -7.59633482e-02  3.81111093e-02 -8.46337806e-03  3.16928737e-02
  3.59484479e-02 -5.85747808e-02  7.93158561e-02  1.30121624e-02
  5.90133220e-02 -3.66817

384

### **1. Vector Search**

In [18]:
 
search_client = SearchClient(service_endpoint,
                             index_name,
                             AzureKeyCredential(key)) 

vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=5, # this varibale defines the number of outputs
                               fields="embedding")

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=5
)  

# results
for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Captions: {result['@search.captions']}")  
    print(f"Highlights: {result['@search.highlights']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.5896371
Captions: None
Highlights: None
Content: 3 Diagrammatic sectional view of :
(a) Coelomate (b) Pseudocoelomate
(c) AcoelomateThose animals in which the developing embryo has a third germinal layer,
mesoderm , in between the ectoderm and endoderm, are called
triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).
4.1.5 Segmentation
In some animals, the body is externally and internally divided into
segments with a serial repetition of at least some organs

Filename: kebo104.pdf

###############################
Score: 0.581004
Captions: None
Highlights: None
Content: Segmentation in the body is first observed in which of the following:
(a) Platyhelminthes  (b) Aschelminthes  (c) Annelida  (d) Arthropoda
14.Match the following:
(a)Operculum (i)Ctenophora
(b)Parapodia (ii)Mollusca
(c)Scales (iii)Porifera
(d)Comb plates (iv)Reptilia
(e)Radula (v)Annelida
(f)Hairs (vi)Cyclostomata and Chondrichthyes
(g)Choanocytes (vii) Mammalia
(h)Gill slits (viii) Osteichthyes
1

### **2. Hybrid Search:**
- Query as well as the vector search 

In [19]:
search_client = SearchClient(service_endpoint,
                             index_name, 
                             AzureKeyCredential(key))  

# The query parameters to use for vector search when a raw vector value is provided
vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, # this varibale defines the number of outputs
                               fields="embedding")


results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)  

  
for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.03333333507180214
Content: 3 Diagrammatic sectional view of :
(a) Coelomate (b) Pseudocoelomate
(c) AcoelomateThose animals in which the developing embryo has a third germinal layer,
mesoderm , in between the ectoderm and endoderm, are called
triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).
4.1.5 Segmentation
In some animals, the body is externally and internally divided into
segments with a serial repetition of at least some organs

Filename: kebo104.pdf

###############################
Score: 0.032522473484277725
Content:  For example, in
earthworm, the body shows this pattern called metameric segmentation
and the phenomenon is known as metamerism .
4.1.6 Notochord
Notochord is a mesodermally derived rod-like structure formed on the
dorsal side during embryonic development in some animals. Animals with
notochord are called chordates and those animals which do not form this
structure are called non-chordates, e.g., porifera to echinoderms.
4

Filename: kebo

### **3. Exhaustive KNN exact nearest neighbor search**

In [20]:
search_client = SearchClient(service_endpoint, 
                             index_name, 
                             AzureKeyCredential(key))  


vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding",
                               exhaustive=True)


results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)  
  

for result in results:  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['line']}\n")  
    print(f"Filename: {result['filename']}\n")  
    print("###############################")

Score: 0.589637
Content: 3 Diagrammatic sectional view of :
(a) Coelomate (b) Pseudocoelomate
(c) AcoelomateThose animals in which the developing embryo has a third germinal layer,
mesoderm , in between the ectoderm and endoderm, are called
triploblastic  animals (platyhelminthes to chordates, Figure 4.2b).
4.1.5 Segmentation
In some animals, the body is externally and internally divided into
segments with a serial repetition of at least some organs

Filename: kebo104.pdf

###############################
Score: 0.581004
Content: Segmentation in the body is first observed in which of the following:
(a) Platyhelminthes  (b) Aschelminthes  (c) Annelida  (d) Arthropoda
14.Match the following:
(a)Operculum (i)Ctenophora
(b)Parapodia (ii)Mollusca
(c)Scales (iii)Porifera
(d)Comb plates (iv)Reptilia
(e)Radula (v)Annelida
(f)Hairs (vi)Cyclostomata and Chondrichthyes
(g)Choanocytes (vii) Mammalia
(h)Gill slits (viii) Osteichthyes
15.Prepare a list of some animals that are found parasitic on huma

### **Semantic Hybrid**

### **Attetion!**
- In order to use semantic search, we need to activate a `sematic ranker` plan which is not available for the free tier accounts!


In [21]:
search_client = SearchClient(service_endpoint, 
                             index_name, 
                             AzureKeyCredential(key))  


vector_query = VectorizedQuery(vector=query_vector, 
                               k_nearest_neighbors=3, 
                               fields="embedding")


results = search_client.search (  
                                search_text=query,  
                                vector_queries=[vector_query],
                                select=["line","filename"],
                                query_type=QueryType.SEMANTIC, 
                                semantic_configuration_name='my-semantic-config', 
                                query_caption=QueryCaptionType.EXTRACTIVE, 
                                query_answer=QueryAnswerType.EXTRACTIVE,
                                top=3
                             )  

In [1]:
semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

In [ ]:
semantic_answers

In [ ]:
for result in results:
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['line']}\n")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")